# Part I. ETL Pipeline for Pre-Processing the Music App History Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [16]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        csvreader = csv.reader(csvfile) 
        next(csvreader)

        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [2]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


### Visualise What The Data Looks Like

In [3]:
data = pd.read_csv('event_datafile_new.csv')
data

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Sydney Youngblood,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",954,Ain't No Sunshine,73
1,Gang Starr,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",984,My Advice 2 You (Explicit),24
2,3OH!3,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",984,My First Kiss (Feat. Ke$ha) [Album Version],24
3,RÃÂ¶yksopp,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",954,The Girl and The Robot,73
4,Kajagoogoo,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",984,Too Shy,24
...,...,...,...,...,...,...,...,...,...,...,...
6815,Girl Talk,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",139,Once again,8
6816,Black Eyed Peas,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",9,Pump It,10
6817,Fall Out Boy,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Nobody Puts Baby In The Corner,26
6818,M.I.A.,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Mango Pickle Down River (With The Wilcannia Mob),26


#### Creating a Cluster

In [17]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [18]:
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS udacity_project WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")
except cassandra.cluster.NoHostAvailable as e:
    print(e)

#### Set Keyspace

In [20]:
try:
    session.set_keyspace('udacity_project')
except cassandra.cluster.NoHostAvailable as e:
    print(e)

### 1.a. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [21]:
## I think the sessionId and itemInSession are unique enough for the purpose of this query.
##Query: SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4;
query = "CREATE TABLE IF NOT EXISTS music_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)

Table created successfully


### 1.b. Inserting Data Into music_library Table

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))

# Just to indicate the the data insertion operation is completed
print("Completed")        

Completed


#### 1.c. Query The Table To Test If The Table Satisfies The Query

In [24]:
## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = "SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

results_df = pd.DataFrame(rows)
results_df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### 2.a. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [25]:
# We will use userId and sessionId as the primary key and itemInSession.
# The query only specify ItemInSession as the clustering column.
# Query: SELECT artist, song, firstName, lastName FROM artist_library WHERE userId = 10 AND sessionId=182;
query = "CREATE TABLE IF NOT EXISTS artist_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)


Table created successfully


### 2.b. Insert Data Into artist_library Table

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
print("Completed")

Completed


### 2.c. Run The Query To See If The Table Satisfies The Query Specification

In [27]:
query = "SELECT artist, song, firstName, lastName FROM artist_library WHERE userId = 10 AND sessionId=182;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

results_df = pd.DataFrame(rows)
results_df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3.a. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [28]:
# Base on the query, the name of the song has to be part of the primary key.
# However, I'm afraid it might not be sufficient to enforce the uniqueness of the row.
# So I also include the userId and sessionId.
# Query: SELECT firstName, lastName FROM song_library WHERE song = 'All Hands Against His Own';
query = "CREATE TABLE IF NOT EXISTS song_library (artist text, firstName text, gender text, itemInSession int, "
query = query + "lastName text, length float, level text, location text, sessionId int, song text, userId int, "
query = query + "PRIMARY KEY (song, userId, sessionId))"
try:
    session.execute(query)
    print("Table created successfully")
except cassandra.cluster.NoHostAvailable as e:
    print(e)


Table created successfully


### 3.b. Insert Data Into song_library Table

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], int(line[8]), line[9], int(line[10])))
        
print("Completed")

Completed


### 3.c. Test If The Query Can Be Run Successfully

In [30]:
query = "SELECT firstName, lastName FROM song_library WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

results_df = pd.DataFrame(rows)
results_df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [31]:
query = "drop table music_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [32]:
query = "drop table artist_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [33]:
query = "drop table song_library;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [34]:
session.shutdown()
cluster.shutdown()